content 로부터 다양한 질문을 생성해서 json 파일로 저장.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI
import os
import json
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
llm_model = "gpt-4o-2024-08-06"
client = OpenAI()

In [3]:
with open("../../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

In [4]:
docs[0]

{'docid': '42508ee0-c543-4338-878e-d98c6babee66',
 'src': 'ko_mmlu__nutrition__test',
 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.'}

In [5]:
augment_instruct = """
## Role
가상 데이터 생성기

## Instructions
- 주어진 레퍼런스 정보를 보고 레퍼런스와 관련 있는 질문을 한국어로 최대 10개 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Output format
{"questions": [$question1, $question2, $question3, $question4, $question5, $question6, $question7, $question8, $question9, $question10]}
"""

In [6]:
def autogen(messages, llm_model):
  result = client.chat.completions.create(
            model=llm_model,
            messages=messages,
            temperature=0.5,
            response_format={"type": "json_object"},
            timeout=10,
            seed=1
    )
  return result

In [8]:
for docOffset in range(4036, len(docs)):
    # if docOffset >= 2: break
    
    doc = docs[docOffset]
    
    messages = [
        {"role": "system", "content": augment_instruct},
        {"role": "user", "content": doc['content']}
    ]

    result = autogen(messages, llm_model)
    
    questions = json.loads(result.choices[0].message.content)['questions']
    print(f'docOffset: {docOffset}, Gen counts: {len(questions)},  Gen questions: {questions}')
    
    if len(questions) <= 0:
        assert False, "hmm..."
    
    dic_list = [
        {
            "question": question,
            "docOffset": docOffset,
        }
        for question in questions
    ]
    
    with open('questions_from_contents.jsonl', 'a') as f:
        for item in dic_list:
            json_line = json.dumps(item, ensure_ascii=False)  # JSON 객체를 문자열로 변환
            f.write(json_line + '\n')     # 각 객체를 한 줄에 작성

docOffset: 4036, Gen counts: 10,  Gen questions: ['세균 효소에서 극성 아미노산이 비극성 아미노산으로 바뀌면 어떤 변화가 발생하나요?', '효소의 활성 부위와 멀리 떨어진 곳의 아미노산 돌연변이가 효소의 기능에 어떤 영향을 미칠 수 있나요?', '효소의 기질 특이성이란 무엇인가요?', '효소의 단백질 형태 변경이 기질 특이성에 미치는 영향은 무엇인가요?', '돌연변이가 효소의 기질 특이성을 어떻게 변경할 수 있나요?', '비극성 아미노산으로의 돌연변이가 효소의 기질 특이성에 미치는 영향은 무엇인가요?', '효소의 활성 부위와 기질 사이의 상호작용 변화는 어떤 결과를 초래할 수 있나요?', '효소가 돌연변이 후에 다양한 기질에 반응할 수 있게 되는 이유는 무엇인가요?', '활성 부위에서 멀리 떨어진 아미노산 돌연변이가 효소의 구조에 미치는 영향은 무엇인가요?', '효소의 기질 특이성이 변경되면 생물학적 과정에서 어떤 변화가 일어날 수 있나요?']
docOffset: 4037, Gen counts: 10,  Gen questions: ['지진 활동이 암석 층에 어떤 영향을 미칠 수 있나요?', '지진으로 인해 더 오래된 암석이 더 어린 암석 위에 위치할 수 있는 이유는 무엇인가요?', '지질학자들은 암석 층의 역전 현상을 통해 어떤 정보를 얻을 수 있나요?', '지진이 발생할 때 암석들이 어떻게 변형되나요?', '지구의 플레이트 움직임이 지진에 어떻게 기여하나요?', '암석 층의 배열이 지질학적 연구에 왜 중요한가요?', '지진 활동이 암석의 층을 역전시키는 과정은 무엇인가요?', '지질학자들이 지구의 지질 변화를 연구할 때 사용하는 방법은 무엇인가요?', '암석 층의 역전 현상이 어떤 지질학적 사건을 나타낼 수 있나요?', '지진 활동으로 인한 암석 층의 역전 현상은 어떻게 식별할 수 있나요?']
docOffset: 4038, Gen counts: 10,  Gen questions: ['중탄산나트륨이 중거리 달리기 성적에 